In [1]:
from keras.optimizers import Adam, SGD
from keras.models import Sequential, Model
from keras import layers
from keras.layers import *
from keras.utils import Sequence
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, CSVLogger, ModelCheckpoint
from keras.engine.topology import get_source_inputs
from keras.utils import layer_utils
from keras import backend as K
from keras.utils.data_utils import get_file
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import os
import datetime
import gc
import pickle
from matplotlib import pyplot as plt
from sklearn.model_selection import StratifiedKFold
from keras.backend import clear_session
import tensorflow as tf
import random as python_random

In [2]:
tf.get_logger().setLevel('WARNING')

# set seeds
np.random.seed(7654)
python_random.seed(7654)
tf.random.set_seed(7654)

In [3]:
WEIGHTS_NAME = ['rgb_kinetics_only', 'flow_kinetics_only', 'rgb_imagenet_and_kinetics', 'flow_imagenet_and_kinetics']

In [4]:
# path to pretrained models with top (classification layer)
WEIGHTS_PATH = {
    'rgb_kinetics_only' : 'https://github.com/dlpbc/keras-kinetics-i3d/releases/download/v0.2/rgb_inception_i3d_kinetics_only_tf_dim_ordering_tf_kernels.h5',
    'flow_kinetics_only' : 'https://github.com/dlpbc/keras-kinetics-i3d/releases/download/v0.2/flow_inception_i3d_kinetics_only_tf_dim_ordering_tf_kernels.h5',
    'rgb_imagenet_and_kinetics' : 'https://github.com/dlpbc/keras-kinetics-i3d/releases/download/v0.2/rgb_inception_i3d_imagenet_and_kinetics_tf_dim_ordering_tf_kernels.h5',
    'flow_imagenet_and_kinetics' : 'https://github.com/dlpbc/keras-kinetics-i3d/releases/download/v0.2/flow_inception_i3d_imagenet_and_kinetics_tf_dim_ordering_tf_kernels.h5'
}

# path to pretrained models with no top (no classification layer)
WEIGHTS_PATH_NO_TOP = {
    'rgb_kinetics_only' : 'https://github.com/dlpbc/keras-kinetics-i3d/releases/download/v0.2/rgb_inception_i3d_kinetics_only_tf_dim_ordering_tf_kernels_no_top.h5',
    'flow_kinetics_only' : 'https://github.com/dlpbc/keras-kinetics-i3d/releases/download/v0.2/flow_inception_i3d_kinetics_only_tf_dim_ordering_tf_kernels_no_top.h5',
    'rgb_imagenet_and_kinetics' : 'https://github.com/dlpbc/keras-kinetics-i3d/releases/download/v0.2/rgb_inception_i3d_imagenet_and_kinetics_tf_dim_ordering_tf_kernels_no_top.h5',
    'flow_imagenet_and_kinetics' : 'https://github.com/dlpbc/keras-kinetics-i3d/releases/download/v0.2/flow_inception_i3d_imagenet_and_kinetics_tf_dim_ordering_tf_kernels_no_top.h5'
}

In [5]:
def _obtain_input_shape(input_shape,
                        default_frame_size,
                        min_frame_size,
                        default_num_frames,
                        min_num_frames,
                        data_format,
                        require_flatten,
                        weights=None):
    """Internal utility to compute/validate the model's input shape.
    (Adapted from `keras/applications/imagenet_utils.py`)
    # Arguments
        input_shape: either None (will return the default network input shape),
            or a user-provided shape to be validated.
        default_frame_size: default input frames(images) width/height for the model.
        min_frame_size: minimum input frames(images) width/height accepted by the model.
        default_num_frames: default input number of frames(images) for the model.
        min_num_frames: minimum input number of frames accepted by the model.
        data_format: image data format to use.
        require_flatten: whether the model is expected to
            be linked to a classifier via a Flatten layer.
        weights: one of `None` (random initialization)
            or 'kinetics_only' (pre-training on Kinetics dataset).
            or 'imagenet_and_kinetics' (pre-training on ImageNet and Kinetics datasets).
            If weights='kinetics_only' or weights=='imagenet_and_kinetics' then
            input channels must be equal to 3.
    # Returns
        An integer shape tuple (may include None entries).
    # Raises
        ValueError: in case of invalid argument values.
    """
    if weights != 'kinetics_only' and weights != 'imagenet_and_kinetics' and input_shape and len(input_shape) == 4:
        if data_format == 'channels_first':
            if input_shape[0] not in {1, 3}:
                warnings.warn(
                    'This model usually expects 1 or 3 input channels. '
                    'However, it was passed an input_shape with ' +
                    str(input_shape[0]) + ' input channels.')
            default_shape = (input_shape[0], default_num_frames, default_frame_size, default_frame_size)
        else:
            if input_shape[-1] not in {1, 3}:
                warnings.warn(
                    'This model usually expects 1 or 3 input channels. '
                    'However, it was passed an input_shape with ' +
                    str(input_shape[-1]) + ' input channels.')
            default_shape = (default_num_frames, default_frame_size, default_frame_size, input_shape[-1])
    else:
        if data_format == 'channels_first':
            default_shape = (3, default_num_frames, default_frame_size, default_frame_size)
        else:
            default_shape = (default_num_frames, default_frame_size, default_frame_size, 3)
    if (weights == 'kinetics_only' or weights == 'imagenet_and_kinetics') and require_flatten:
        if input_shape is not None:
            if input_shape != default_shape:
                raise ValueError('When setting`include_top=True` '
                                 'and loading `imagenet` weights, '
                                 '`input_shape` should be ' +
                                 str(default_shape) + '.')
        return default_shape

    if input_shape:
        if data_format == 'channels_first':
            if input_shape is not None:
                if len(input_shape) != 4:
                    raise ValueError(
                        '`input_shape` must be a tuple of four integers.')
                if input_shape[0] != 3 and (weights == 'kinetics_only' or weights == 'imagenet_and_kinetics'):
                    raise ValueError('The input must have 3 channels; got '
                                     '`input_shape=' + str(input_shape) + '`')

                if input_shape[1] is not None and input_shape[1] < min_num_frames:
                    raise ValueError('Input number of frames must be at least ' +
                                     str(min_num_frames) + '; got '
                                     '`input_shape=' + str(input_shape) + '`')

                if ((input_shape[2] is not None and input_shape[2] < min_frame_size) or
                   (input_shape[3] is not None and input_shape[3] < min_frame_size)):
                    raise ValueError('Input size must be at least ' +
                                     str(min_frame_size) + 'x' + str(min_frame_size) + '; got '
                                     '`input_shape=' + str(input_shape) + '`')
        else:
            if input_shape is not None:
                if len(input_shape) != 4:
                    raise ValueError(
                        '`input_shape` must be a tuple of four integers.')
                if input_shape[-1] != 3 and (weights == 'kinetics_only' or weights == 'imagenet_and_kinetics'):
                    raise ValueError('The input must have 3 channels; got '
                                     '`input_shape=' + str(input_shape) + '`')

                if input_shape[0] is not None and input_shape[0] < min_num_frames:
                    raise ValueError('Input number of frames must be at least ' +
                                     str(min_num_frames) + '; got '
                                     '`input_shape=' + str(input_shape) + '`')

                if ((input_shape[1] is not None and input_shape[1] < min_frame_size) or
                   (input_shape[2] is not None and input_shape[2] < min_frame_size)):
                    raise ValueError('Input size must be at least ' +
                                     str(min_frame_size) + 'x' + str(min_frame_size) + '; got '
                                     '`input_shape=' + str(input_shape) + '`')
    else:
        if require_flatten:
            input_shape = default_shape
        else:
            if data_format == 'channels_first':
                input_shape = (3, None, None, None)
            else:
                input_shape = (None, None, None, 3)
    if require_flatten:
        if None in input_shape:
            raise ValueError('If `include_top` is True, '
                             'you should specify a static `input_shape`. '
                             'Got `input_shape=' + str(input_shape) + '`')
    return input_shape


In [6]:
def conv3d_bn(x,
              filters,
              num_frames,
              num_row,
              num_col,
              padding='same',
              strides=(1, 1, 1),
              use_bias = False,
              use_activation_fn = True,
              use_bn = True,
              name=None):
    """Utility function to apply conv3d + BN.
    # Arguments
        x: input tensor.
        filters: filters in `Conv3D`.
        num_frames: frames (time depth) of the convolution kernel.
        num_row: height of the convolution kernel.
        num_col: width of the convolution kernel.
        padding: padding mode in `Conv3D`.
        strides: strides in `Conv3D`.
        use_bias: use bias or not  
        use_activation_fn: use an activation function or not.
        use_bn: use batch normalization or not.
        name: name of the ops; will become `name + '_conv'`
            for the convolution and `name + '_bn'` for the
            batch norm layer.
    # Returns
        Output tensor after applying `Conv3D` and `BatchNormalization`.
    """
    if name is not None:
        bn_name = name + '_bn'
        conv_name = name + '_conv'
    else:
        bn_name = None
        conv_name = None

    x = Conv3D(
        filters, (num_frames, num_row, num_col),
        strides=strides,
        padding=padding,
        use_bias=use_bias,
        name=conv_name)(x)

    if use_bn:
        if K.image_data_format() == 'channels_first':
            bn_axis = 1
        else:
            bn_axis = 4
        x = BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(x)

    if use_activation_fn:
        x = Activation('relu', name=name)(x)

    return x

In [7]:
def Inception_Inflated3d(include_top=True,
                weights=None,
                input_tensor=None,
                input_shape=None,
                dropout_prob=0.0,
                endpoint_logit=True,
                classes=400):
    """Instantiates the Inflated 3D Inception v1 architecture.
    Optionally loads weights pre-trained
    on Kinetics. Note that when using TensorFlow,
    for best performance you should set
    `image_data_format='channels_last'` in your Keras config
    at ~/.keras/keras.json.
    The model and the weights are compatible with both
    TensorFlow and Theano. The data format
    convention used by the model is the one
    specified in your Keras config file.
    Note that the default input frame(image) size for this model is 224x224.
    # Arguments
        include_top: whether to include the the classification 
            layer at the top of the network.
        weights: one of `None` (random initialization)
            or 'kinetics_only' (pre-training on Kinetics dataset only).
            or 'imagenet_and_kinetics' (pre-training on ImageNet and Kinetics datasets).
        input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
            to use as image input for the model.
        input_shape: optional shape tuple, only to be specified
            if `include_top` is False (otherwise the input shape
            has to be `(NUM_FRAMES, 224, 224, 3)` (with `channels_last` data format)
            or `(NUM_FRAMES, 3, 224, 224)` (with `channels_first` data format).
            It should have exactly 3 inputs channels.
            NUM_FRAMES should be no smaller than 8. The authors used 64
            frames per example for training and testing on kinetics dataset
            Also, Width and height should be no smaller than 32.
            E.g. `(64, 150, 150, 3)` would be one valid value.
        dropout_prob: optional, dropout probability applied in dropout layer
            after global average pooling layer. 
            0.0 means no dropout is applied, 1.0 means dropout is applied to all features.
            Note: Since Dropout is applied just before the classification
            layer, it is only useful when `include_top` is set to True.
        endpoint_logit: (boolean) optional. If True, the model's forward pass
            will end at producing logits. Otherwise, softmax is applied after producing
            the logits to produce the class probabilities prediction. Setting this parameter 
            to True is particularly useful when you want to combine results of rgb model
            and optical flow model.
            - `True` end model forward pass at logit output
            - `False` go further after logit to produce softmax predictions
            Note: This parameter is only useful when `include_top` is set to True.
        classes: optional number of classes to classify images
            into, only to be specified if `include_top` is True, and
            if no `weights` argument is specified.
    # Returns
        A Keras model instance.
    # Raises
        ValueError: in case of invalid argument for `weights`,
            or invalid input shape.
    """
    if not (weights in WEIGHTS_NAME or weights is None or os.path.exists(weights)):
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization) or %s' % 
                         str(WEIGHTS_NAME) + ' ' 
                         'or a valid path to a file containing `weights` values')

    if weights in WEIGHTS_NAME and include_top and classes != 400:
        raise ValueError('If using `weights` as one of these %s, with `include_top`'
                         ' as true, `classes` should be 400' % str(WEIGHTS_NAME))

    # Determine proper input shape
    input_shape = _obtain_input_shape(
        input_shape,
        default_frame_size=224, 
        min_frame_size=32, 
        default_num_frames=64,
        min_num_frames=8,
        data_format=K.image_data_format(),
        require_flatten=include_top,
        weights=weights)

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor

    if K.image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = 4

    # Downsampling via convolution (spatial and temporal)
    x = conv3d_bn(img_input, 64, 7, 7, 7, strides=(2, 2, 2), padding='same', name='Conv3d_1a_7x7')

    # Downsampling (spatial only)
    x = MaxPooling3D((1, 3, 3), strides=(1, 2, 2), padding='same', name='MaxPool2d_2a_3x3')(x)
    x = conv3d_bn(x, 64, 1, 1, 1, strides=(1, 1, 1), padding='same', name='Conv3d_2b_1x1')
    x = conv3d_bn(x, 192, 3, 3, 3, strides=(1, 1, 1), padding='same', name='Conv3d_2c_3x3')

    # Downsampling (spatial only)
    x = MaxPooling3D((1, 3, 3), strides=(1, 2, 2), padding='same', name='MaxPool2d_3a_3x3')(x)

    # Mixed 3b
    branch_0 = conv3d_bn(x, 64, 1, 1, 1, padding='same', name='Conv3d_3b_0a_1x1')

    branch_1 = conv3d_bn(x, 96, 1, 1, 1, padding='same', name='Conv3d_3b_1a_1x1')
    branch_1 = conv3d_bn(branch_1, 128, 3, 3, 3, padding='same', name='Conv3d_3b_1b_3x3')

    branch_2 = conv3d_bn(x, 16, 1, 1, 1, padding='same', name='Conv3d_3b_2a_1x1')
    branch_2 = conv3d_bn(branch_2, 32, 3, 3, 3, padding='same', name='Conv3d_3b_2b_3x3')

    branch_3 = MaxPooling3D((3, 3, 3), strides=(1, 1, 1), padding='same', name='MaxPool2d_3b_3a_3x3')(x)
    branch_3 = conv3d_bn(branch_3, 32, 1, 1, 1, padding='same', name='Conv3d_3b_3b_1x1')

    x = layers.concatenate(
        [branch_0, branch_1, branch_2, branch_3],
        axis=channel_axis,
        name='Mixed_3b')

    # Mixed 3c
    branch_0 = conv3d_bn(x, 128, 1, 1, 1, padding='same', name='Conv3d_3c_0a_1x1')

    branch_1 = conv3d_bn(x, 128, 1, 1, 1, padding='same', name='Conv3d_3c_1a_1x1')
    branch_1 = conv3d_bn(branch_1, 192, 3, 3, 3, padding='same', name='Conv3d_3c_1b_3x3')

    branch_2 = conv3d_bn(x, 32, 1, 1, 1, padding='same', name='Conv3d_3c_2a_1x1')
    branch_2 = conv3d_bn(branch_2, 96, 3, 3, 3, padding='same', name='Conv3d_3c_2b_3x3')

    branch_3 = MaxPooling3D((3, 3, 3), strides=(1, 1, 1), padding='same', name='MaxPool2d_3c_3a_3x3')(x)
    branch_3 = conv3d_bn(branch_3, 64, 1, 1, 1, padding='same', name='Conv3d_3c_3b_1x1')

    x = layers.concatenate(
        [branch_0, branch_1, branch_2, branch_3],
        axis=channel_axis,
        name='Mixed_3c')


    # Downsampling (spatial and temporal)
    x = MaxPooling3D((3, 3, 3), strides=(2, 2, 2), padding='same', name='MaxPool2d_4a_3x3')(x)

    # Mixed 4b
    branch_0 = conv3d_bn(x, 192, 1, 1, 1, padding='same', name='Conv3d_4b_0a_1x1')

    branch_1 = conv3d_bn(x, 96, 1, 1, 1, padding='same', name='Conv3d_4b_1a_1x1')
    branch_1 = conv3d_bn(branch_1, 208, 3, 3, 3, padding='same', name='Conv3d_4b_1b_3x3')

    branch_2 = conv3d_bn(x, 16, 1, 1, 1, padding='same', name='Conv3d_4b_2a_1x1')
    branch_2 = conv3d_bn(branch_2, 48, 3, 3, 3, padding='same', name='Conv3d_4b_2b_3x3')

    branch_3 = MaxPooling3D((3, 3, 3), strides=(1, 1, 1), padding='same', name='MaxPool2d_4b_3a_3x3')(x)
    branch_3 = conv3d_bn(branch_3, 64, 1, 1, 1, padding='same', name='Conv3d_4b_3b_1x1')

    x = layers.concatenate(
        [branch_0, branch_1, branch_2, branch_3],
        axis=channel_axis,
        name='Mixed_4b')

    # Mixed 4c
    branch_0 = conv3d_bn(x, 160, 1, 1, 1, padding='same', name='Conv3d_4c_0a_1x1')

    branch_1 = conv3d_bn(x, 112, 1, 1, 1, padding='same', name='Conv3d_4c_1a_1x1')
    branch_1 = conv3d_bn(branch_1, 224, 3, 3, 3, padding='same', name='Conv3d_4c_1b_3x3')

    branch_2 = conv3d_bn(x, 24, 1, 1, 1, padding='same', name='Conv3d_4c_2a_1x1')
    branch_2 = conv3d_bn(branch_2, 64, 3, 3, 3, padding='same', name='Conv3d_4c_2b_3x3')

    branch_3 = MaxPooling3D((3, 3, 3), strides=(1, 1, 1), padding='same', name='MaxPool2d_4c_3a_3x3')(x)
    branch_3 = conv3d_bn(branch_3, 64, 1, 1, 1, padding='same', name='Conv3d_4c_3b_1x1')

    x = layers.concatenate(
        [branch_0, branch_1, branch_2, branch_3],
        axis=channel_axis,
        name='Mixed_4c')

    # Mixed 4d
    branch_0 = conv3d_bn(x, 128, 1, 1, 1, padding='same', name='Conv3d_4d_0a_1x1')

    branch_1 = conv3d_bn(x, 128, 1, 1, 1, padding='same', name='Conv3d_4d_1a_1x1')
    branch_1 = conv3d_bn(branch_1, 256, 3, 3, 3, padding='same', name='Conv3d_4d_1b_3x3')

    branch_2 = conv3d_bn(x, 24, 1, 1, 1, padding='same', name='Conv3d_4d_2a_1x1')
    branch_2 = conv3d_bn(branch_2, 64, 3, 3, 3, padding='same', name='Conv3d_4d_2b_3x3')

    branch_3 = MaxPooling3D((3, 3, 3), strides=(1, 1, 1), padding='same', name='MaxPool2d_4d_3a_3x3')(x)
    branch_3 = conv3d_bn(branch_3, 64, 1, 1, 1, padding='same', name='Conv3d_4d_3b_1x1')

    x = layers.concatenate(
        [branch_0, branch_1, branch_2, branch_3],
        axis=channel_axis,
        name='Mixed_4d')

    # Mixed 4e
    branch_0 = conv3d_bn(x, 112, 1, 1, 1, padding='same', name='Conv3d_4e_0a_1x1')

    branch_1 = conv3d_bn(x, 144, 1, 1, 1, padding='same', name='Conv3d_4e_1a_1x1')
    branch_1 = conv3d_bn(branch_1, 288, 3, 3, 3, padding='same', name='Conv3d_4e_1b_3x3')

    branch_2 = conv3d_bn(x, 32, 1, 1, 1, padding='same', name='Conv3d_4e_2a_1x1')
    branch_2 = conv3d_bn(branch_2, 64, 3, 3, 3, padding='same', name='Conv3d_4e_2b_3x3')

    branch_3 = MaxPooling3D((3, 3, 3), strides=(1, 1, 1), padding='same', name='MaxPool2d_4e_3a_3x3')(x)
    branch_3 = conv3d_bn(branch_3, 64, 1, 1, 1, padding='same', name='Conv3d_4e_3b_1x1')

    x = layers.concatenate(
        [branch_0, branch_1, branch_2, branch_3],
        axis=channel_axis,
        name='Mixed_4e')

    # Mixed 4f
    branch_0 = conv3d_bn(x, 256, 1, 1, 1, padding='same', name='Conv3d_4f_0a_1x1')

    branch_1 = conv3d_bn(x, 160, 1, 1, 1, padding='same', name='Conv3d_4f_1a_1x1')
    branch_1 = conv3d_bn(branch_1, 320, 3, 3, 3, padding='same', name='Conv3d_4f_1b_3x3')

    branch_2 = conv3d_bn(x, 32, 1, 1, 1, padding='same', name='Conv3d_4f_2a_1x1')
    branch_2 = conv3d_bn(branch_2, 128, 3, 3, 3, padding='same', name='Conv3d_4f_2b_3x3')

    branch_3 = MaxPooling3D((3, 3, 3), strides=(1, 1, 1), padding='same', name='MaxPool2d_4f_3a_3x3')(x)
    branch_3 = conv3d_bn(branch_3, 128, 1, 1, 1, padding='same', name='Conv3d_4f_3b_1x1')

    x = layers.concatenate(
        [branch_0, branch_1, branch_2, branch_3],
        axis=channel_axis,
        name='Mixed_4f')


    # Downsampling (spatial and temporal)
    x = MaxPooling3D((2, 2, 2), strides=(2, 2, 2), padding='same', name='MaxPool2d_5a_2x2')(x)

    # Mixed 5b
    branch_0 = conv3d_bn(x, 256, 1, 1, 1, padding='same', name='Conv3d_5b_0a_1x1')

    branch_1 = conv3d_bn(x, 160, 1, 1, 1, padding='same', name='Conv3d_5b_1a_1x1')
    branch_1 = conv3d_bn(branch_1, 320, 3, 3, 3, padding='same', name='Conv3d_5b_1b_3x3')

    branch_2 = conv3d_bn(x, 32, 1, 1, 1, padding='same', name='Conv3d_5b_2a_1x1')
    branch_2 = conv3d_bn(branch_2, 128, 3, 3, 3, padding='same', name='Conv3d_5b_2b_3x3')

    branch_3 = MaxPooling3D((3, 3, 3), strides=(1, 1, 1), padding='same', name='MaxPool2d_5b_3a_3x3')(x)
    branch_3 = conv3d_bn(branch_3, 128, 1, 1, 1, padding='same', name='Conv3d_5b_3b_1x1')

    x = layers.concatenate(
        [branch_0, branch_1, branch_2, branch_3],
        axis=channel_axis,
        name='Mixed_5b')

    # Mixed 5c
    branch_0 = conv3d_bn(x, 384, 1, 1, 1, padding='same', name='Conv3d_5c_0a_1x1')

    branch_1 = conv3d_bn(x, 192, 1, 1, 1, padding='same', name='Conv3d_5c_1a_1x1')
    branch_1 = conv3d_bn(branch_1, 384, 3, 3, 3, padding='same', name='Conv3d_5c_1b_3x3')

    branch_2 = conv3d_bn(x, 48, 1, 1, 1, padding='same', name='Conv3d_5c_2a_1x1')
    branch_2 = conv3d_bn(branch_2, 128, 3, 3, 3, padding='same', name='Conv3d_5c_2b_3x3')

    branch_3 = MaxPooling3D((3, 3, 3), strides=(1, 1, 1), padding='same', name='MaxPool2d_5c_3a_3x3')(x)
    branch_3 = conv3d_bn(branch_3, 128, 1, 1, 1, padding='same', name='Conv3d_5c_3b_1x1')

    x = layers.concatenate(
        [branch_0, branch_1, branch_2, branch_3],
        axis=channel_axis,
        name='Mixed_5c')

    if include_top:
        # Classification block
        x = AveragePooling3D((2, 7, 7), strides=(1, 1, 1), padding='valid', name='global_avg_pool')(x)
        x = Dropout(dropout_prob)(x)

        x = conv3d_bn(x, classes, 1, 1, 1, padding='same', 
                use_bias=True, use_activation_fn=False, use_bn=False, name='Conv3d_6a_1x1')
 
        num_frames_remaining = int(x.shape[1])
        x = Reshape((num_frames_remaining, classes))(x)

        # logits (raw scores for each class)
        x = Lambda(lambda x: K.mean(x, axis=1, keepdims=False),
                   output_shape=lambda s: (s[0], s[2]))(x)

        if not endpoint_logit:
            x = Activation('softmax', name='prediction')(x)
    else:
        h = int(x.shape[2])
        w = int(x.shape[3])
        x = AveragePooling3D((2, h, w), strides=(1, 1, 1), padding='valid', name='global_avg_pool')(x)



    inputs = img_input
    # create model
    model = Model(inputs, x, name='i3d_inception')

    # load weights
    if weights in WEIGHTS_NAME:
        if weights == WEIGHTS_NAME[0]:   # rgb_kinetics_only
            if include_top:
                weights_url = WEIGHTS_PATH['rgb_kinetics_only']
                model_name = 'i3d_inception_rgb_kinetics_only.h5'
            else:
                weights_url = WEIGHTS_PATH_NO_TOP['rgb_kinetics_only']
                model_name = 'i3d_inception_rgb_kinetics_only_no_top.h5'

        elif weights == WEIGHTS_NAME[1]: # flow_kinetics_only
            if include_top:
                weights_url = WEIGHTS_PATH['flow_kinetics_only']
                model_name = 'i3d_inception_flow_kinetics_only.h5'
            else:
                weights_url = WEIGHTS_PATH_NO_TOP['flow_kinetics_only']
                model_name = 'i3d_inception_flow_kinetics_only_no_top.h5'

        elif weights == WEIGHTS_NAME[2]: # rgb_imagenet_and_kinetics
            if include_top:
                weights_url = WEIGHTS_PATH['rgb_imagenet_and_kinetics']
                model_name = 'i3d_inception_rgb_imagenet_and_kinetics.h5'
            else:
                weights_url = WEIGHTS_PATH_NO_TOP['rgb_imagenet_and_kinetics']
                model_name = 'i3d_inception_rgb_imagenet_and_kinetics_no_top.h5'

        elif weights == WEIGHTS_NAME[3]: # flow_imagenet_and_kinetics
            if include_top:
                weights_url = WEIGHTS_PATH['flow_imagenet_and_kinetics']
                model_name = 'i3d_inception_flow_imagenet_and_kinetics.h5'
            else:
                weights_url = WEIGHTS_PATH_NO_TOP['flow_imagenet_and_kinetics']
                model_name = 'i3d_inception_flow_imagenet_and_kinetics_no_top.h5'

        downloaded_weights_path = get_file(model_name, weights_url, cache_subdir='models')
        model.load_weights(downloaded_weights_path)

        if K.backend() == 'theano':
            layer_utils.convert_all_kernels_in_model(model)

        if K.image_data_format() == 'channels_first' and K.backend() == 'tensorflow':
            warnings.warn('You are using the TensorFlow backend, yet you '
                          'are using the Theano '
                          'image data format convention '
                          '(`image_data_format="channels_first"`). '
                          'For best performance, set '
                          '`image_data_format="channels_last"` in '
                          'your keras config '
                          'at ~/.keras/keras.json.')

    elif weights is not None:
        model.load_weights(weights)

    return model

In [8]:
def load(name):
    X = np.load("C:/Users/Saahil/OneDrive/Documents/COMPSCI 760/Research Project/cacophony-preprocessed" + name + ".npy")
    y = np.load("C:/Users/Saahil/OneDrive/Documents/COMPSCI 760/Research Project/cacophony-preprocessed" + name + "-labels.npy")
    y_one_hot_encoded = np.zeros([y.shape[0], np.unique(y).size])
    y_one_hot_encoded[range(y.shape[0]), y] = 1
    return X, y_one_hot_encoded

In [9]:
def define_joint_model(upscaling, d_1, d_2, d_3, f_1, r_1, r_2, pooling):

    MLP = Sequential()
    MLP.add(Flatten())
    MLP.add(Dropout(d_1))
    MLP.add(Dense(f_1, activation = "relu"))
    MLP.add(Dense(13, activation="softmax"))

    vid_inputs = Input((45, upscaling, upscaling, 3))
    mvm_inputs = Input((45, 9))

    # CNN extracts 512 video features for each frame
    x = Inception_Inflated3d(include_top = False,
                         weights = 'rgb_imagenet_and_kinetics',
                         input_shape = (45, upscaling, upscaling, 3))(vid_inputs)

    x = TimeDistributed(Flatten())(x)
  
    vid_features = TimeDistributed(Flatten())(x)

    # LSTM extracts 512 movement features for each frame
    mvm_features = LSTM(r_1, return_sequences=True, dropout = d_2, recurrent_dropout = 0.0)(mvm_inputs)
    if pooling == 'avg':
        mvm_features = AveragePooling1D(pool_size=1, strides=9)(mvm_features)
    elif pooling == 'max':
        mvm_features = MaxPooling1D(pool_size=1, strides=9)(mvm_features)
    # Concatenating for 1024 features for each frame
    x = Concatenate()([vid_features, mvm_features])
    # LSTM across both image and movement data
    x = LSTM(r_2, return_sequences = True, dropout = d_3, recurrent_dropout = 0.0)(x)
    # MLP makes a classification for each frame
    x = TimeDistributed(MLP)(x)
    # Outputting the mean classification of all frames
    outputs = GlobalAveragePooling1D()(x)

    model = Model(inputs=[vid_inputs, mvm_inputs], outputs=outputs)

    return model

In [10]:
class DataGenerator(Sequence):
    def __init__(self, vids, mvm, labels, batch_size, upscaling, flip = False, angle = 0, crop = 0, shift = 0, shuffle = True):
        self.vids = vids
        self.mvm = mvm
        self.labels = labels
        self.indices = np.arange(vids.shape[0])
        self.batch_size = batch_size
        self.flip = flip
        self.angle = angle
        self.crop = crop
        self.shift = shift
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return len(self.indices) // self.batch_size

    def random_zoom(self, batch, x, y):
        ax = np.random.uniform(self.crop)
        bx = np.random.uniform(ax)
        ay = np.random.uniform(self.crop)
        by = np.random.uniform(ay)
        x = x*(1-ax/batch.shape[2]) + bx
        y = y*(1-ay/batch.shape[3]) + by
        return x, y

    def random_rotate(self, batch, x, y):
        rad = np.random.uniform(-self.angle, self.angle)/180*np.pi
        rotm = np.array([[np.cos(rad),  np.sin(rad)],
                         [-np.sin(rad), np.cos(rad)]])
        xm, ym = x.mean(), y.mean()
        x, y = np.einsum('ji, mni -> jmn', rotm, np.dstack([x-xm, y-ym]))
        return x+xm, y+ym

    def random_translate(self, batch, x, y):
        xs = np.random.uniform(-self.shift, self.shift)
        ys = np.random.uniform(-self.shift, self.shift)
        return x + xs, y + ys

    def horizontal_flip(self, batch):
        return np.flip(batch, 3)

    def __getitem__(self, index):
        indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        vids = np.array(self.vids[indices])
        #x, y = np.meshgrid(range(vids.shape[2]), range(vids.shape[3]))
        x, y = np.meshgrid(np.arange(upscaling)*(24 / upscaling), np.arange(upscaling)*(24 / upscaling))
        if self.crop:
            x, y = self.random_zoom(vids, x, y)
        if self.angle:
            x, y = self.random_rotate(vids, x, y)
        if self.shift:
            x, y = self.random_translate(vids, x, y)
        if self.flip and np.random.random() < 0.5:
            vids = self.horizontal_flip(vids)
        x = np.clip(x, 0, vids.shape[2]-1).astype(np.int)
        y = np.clip(y, 0, vids.shape[3]-1).astype(np.int)
        vids = vids[:,:,x,y].transpose(0,1,3,2,4)
        if self.mvm is not None:
            out = [vids, self.mvm[indices]], self.labels[indices]
        else:
            out = vids, self.labels[indices]
        return out
    
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

In [11]:
def plots(history):

    # plot training history
    # two plots
    fig, (ax1, ax2) = plt.subplots(2, 1)

    ax1.plot(history.history['accuracy'])
    ax1.plot(history.history['val_accuracy'])
    ax1.set_title('model accuracy')
    ax1.set_ylabel('accuracy')
    ax1.set_xlabel('epoch')
    ax1.legend(['train', 'val'], loc='upper left')

    ax2.plot(history.history['loss'])
    ax2.plot(history.history['val_loss'])
    ax2.set_title('model loss')
    ax2.set_ylabel('loss')
    ax2.set_xlabel('epoch')
    ax2.legend(['train', 'val'], loc='upper left')

    fig.savefig('./logs/plot' + current_time + '.svg', format = 'svg')

In [12]:
print("Dataset loading..", end = " ")
# Loading the preprocessed videos
X_train, y_train = load("/training")
X_val, y_val = load("/validation")
X_test, y_test = load("/test")

Dataset loading.. 

In [13]:
# Since Keras likes the channels last data format
X_train = X_train.transpose(0,1,3,4,2)
X_val = X_val.transpose(0,1,3,4,2)
X_test = X_test.transpose(0,1,3,4,2)

In [14]:
# Loading the preprocessed movement features
X_train_mvm, _ = load("-movement/training")
X_val_mvm, _ = load("-movement/validation")
X_test_mvm, _ = load("-movement/test")
print("Dataset loaded!")

Dataset loaded!


In [15]:
# for k-fold we want to use both train and validation data to generate the folds
# holdout stays apart
X_train2 = np.concatenate((X_train, X_val))
X_train_mvm2 = np.concatenate((X_train_mvm, X_val_mvm))
y_train2 = np.concatenate((y_train, y_val))

In [16]:
current_time = str(datetime.datetime.now())

In [17]:
# create log dir
if not os.path.exists("./logs/kfold"):
    os.makedirs("./logs/kfold")

if not os.path.exists("./logs/kfold_plotdata"):
    os.makedirs("./logs/kfold_plotdata")

In [18]:
epochs = 100
batch_size = 32
learning_rate = 0.001
upscaling = 32 # from 24x24

kfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 12345)

In [19]:
def get_model(key):
    models = dict()
    models['i3d-1'] = define_joint_model(upscaling, d_1 = 0.5, d_2 = 0.2, d_3 = 0.2, f_1 = 256, r_1 = 512, r_2 = 512, pooling = 'avg')
    models['i3d-2'] = define_joint_model(upscaling, d_1 = 0.2, d_2 = 0.2, d_3 = 0.2, f_1 = 256, r_1 = 512, r_2 = 512, pooling = 'avg')
    models['i3d-3'] = define_joint_model(upscaling, d_1 = 0.5, d_2 = 0.1, d_3 = 0.1, f_1 = 256, r_1 = 512, r_2 = 512, pooling = 'avg')
    models['i3d-4'] = define_joint_model(upscaling, d_1 = 0.2, d_2 = 0.1, d_3 = 0.1, f_1 = 256, r_1 = 512, r_2 = 512, pooling = 'avg')
    models['i3d-5'] = define_joint_model(upscaling, d_1 = 0.5, d_2 = 0.2, d_3 = 0.2, f_1 = 256, r_1 = 512, r_2 = 512, pooling = 'max')
    models['i3d-6'] = define_joint_model(upscaling, d_1 = 0.2, d_2 = 0.2, d_3 = 0.2, f_1 = 256, r_1 = 512, r_2 = 512, pooling = 'max')
    models['i3d-7'] = define_joint_model(upscaling, d_1 = 0.5, d_2 = 0.1, d_3 = 0.1, f_1 = 256, r_1 = 512, r_2 = 512, pooling = 'max')
    models['i3d-8'] = define_joint_model(upscaling, d_1 = 0.2, d_2 = 0.1, d_3 = 0.1, f_1 = 256, r_1 = 512, r_2 = 512, pooling = 'max')
    models['i3d-9'] = define_joint_model(upscaling, d_1 = 0.5, d_2 = 0.2, d_3 = 0.2, f_1 = 512, r_1 = 512, r_2 = 512, pooling = 'avg')
    models['i3d-10'] = define_joint_model(upscaling, d_1 = 0.2, d_2 = 0.2, d_3 = 0.2, f_1 = 512, r_1 = 512, r_2 = 512, pooling = 'avg')
    models['i3d-11'] = define_joint_model(upscaling, d_1 = 0.5, d_2 = 0.1, d_3 = 0.1, f_1 = 512, r_1 = 512, r_2 = 512, pooling = 'avg')
    models['i3d-12'] = define_joint_model(upscaling, d_1 = 0.2, d_2 = 0.1, d_3 = 0.1, f_1 = 512, r_1 = 512, r_2 = 512, pooling = 'avg')
    models['i3d-13'] = define_joint_model(upscaling, d_1 = 0.5, d_2 = 0.2, d_3 = 0.2, f_1 = 512, r_1 = 512, r_2 = 512, pooling = 'max')
    models['i3d-14'] = define_joint_model(upscaling, d_1 = 0.2, d_2 = 0.2, d_3 = 0.2, f_1 = 512, r_1 = 512, r_2 = 512, pooling = 'max')
    models['i3d-15'] = define_joint_model(upscaling, d_1 = 0.5, d_2 = 0.1, d_3 = 0.1, f_1 = 512, r_1 = 512, r_2 = 512, pooling = 'max')
    models['i3d-16'] = define_joint_model(upscaling, d_1 = 0.2, d_2 = 0.1, d_3 = 0.1, f_1 = 512, r_1 = 512, r_2 = 512, pooling = 'max')

    models['i3d-17'] = define_joint_model(upscaling, d_1 = 0.3, d_2 = 0.3, d_3 = 0.3, f_1 = 256, r_1 = 512, r_2 = 512, pooling = 'avg')
    models['i3d-18'] = define_joint_model(upscaling, d_1 = 0.2, d_2 = 0.2, d_3 = 0.2, f_1 = 256, r_1 = 256, r_2 = 256, pooling = 'avg')
    models['i3d-19'] = define_joint_model(upscaling, d_1 = 0.2, d_2 = 0.2, d_3 = 0.2, f_1 = 256, r_1 = 512, r_2 = 256, pooling = 'avg')
    models['i3d-20'] = define_joint_model(upscaling, d_1 = 0.5, d_2 = 0.2, d_3 = 0.2, f_1 = 512, r_1 = 512, r_2 = 512, pooling = 'avg')
    models['i3d-21'] = define_joint_model(upscaling, d_1 = 0.6, d_2 = 0.3, d_3 = 0.3, f_1 = 512, r_1 = 512, r_2 = 1024, pooling = 'avg')
    models['i3d-22'] = define_joint_model(upscaling, d_1 = 0.2, d_2 = 0.2, d_3 = 0.2, f_1 = 256, r_1 = 512, r_2 = 1024, pooling = 'avg')
    models['i3d-23'] = define_joint_model(upscaling, d_1 = 0.1, d_2 = 0.1, d_3 = 0.1, f_1 = 128, r_1 = 256, r_2 = 256, pooling = 'avg')
    models['i3d-24'] = define_joint_model(upscaling, d_1 = 0.2, d_2 = 0.2, d_3 = 0.2, f_1 = 128, r_1 = 256, r_2 = 256, pooling = 'avg')
    models['i3d-25'] = define_joint_model(upscaling, d_1 = 0.4, d_2 = 0.2, d_3 = 0.2, f_1 = 256, r_1 = 512, r_2 = 512, pooling = 'avg')
    return models[key]

In [20]:
def evaluate_model(model, name):
    model_accuracy = []
    model_loss = []
    for fold, (train_idx, val_idx) in enumerate(kfold.split(X = np.zeros(X_train2.shape[0]), y = y_train2.argmax(1))): # because split returns incidies we only need one
        # csv logs based on the time
        model.compile(loss='categorical_crossentropy', optimizer = Adam(lr = learning_rate), metrics=["accuracy"])
        #print(model.summary())
        csv_logger = CSVLogger('./logs/kfold/log_' + str(fold + 1) +'_'+name + '.csv', append=True, separator=';')
        reduce_lr = ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.5, patience = 3, min_lr = 0.00001, verbose = 1)
        checkpointer = ModelCheckpoint(filepath='./logs/kfold/best_weights_fold' + str(fold + 1) +'_'+name + '.hdf5', verbose=1, save_best_only=True, monitor='val_accuracy', mode = 'max')
        callbacks = [EarlyStopping(patience = 10), reduce_lr, csv_logger, checkpointer]
    
        # get data
        X_train, X_train_mvm, y_train = X_train2[train_idx], X_train_mvm2[train_idx], y_train2[train_idx]
        X_val, X_val_mvm, y_val = X_train2[val_idx], X_train_mvm2[val_idx], y_train2[val_idx]
        train_data = DataGenerator(X_train, X_train_mvm, y_train, batch_size, upscaling, True, 10, 0, 0)
        val_data = DataGenerator(X_val, X_val_mvm, y_val, batch_size, upscaling)
        #test_data = DataGenerator(X_test, X_test_mvm, y_test, batch_size, upscaling)
    #    data_x.extend(X_val)
    #    data_mvm.extend(X_val_mvm)
    #    data_y.extend(y_val)
        # fit model
        print('Training fold ' + str(fold + 1) + ' of ' + name + ' with augmentation...')
    #   model = define_model_LRCN()
        model.save_weights('model.h5')       
        history = model.fit(train_data,
              epochs = epochs,
              validation_data = val_data,
              callbacks = callbacks,
              verbose = 0)
        best_epoch = np.argmax(history.history['val_accuracy'])
        model_accuracy.append(history.history['val_accuracy'][best_epoch])
        model_loss.append(history.history['val_loss'][best_epoch])
        model.load_weights('model.h5')
        #preds = model.predict([X_val, X_val_mvm], batch_size = batch_size)[:, 0]
        #preds_lrcn.extend(preds)
        plots(history)
    del model
    clear_session()
    #tf.keras.backend.clear_session()
    return np.asarray(model_accuracy).flatten(), np.asarray(model_loss).flatten()

In [21]:
box_acc, box_loss, names = list(), list(), list()

In [ ]:
for name in ['i3d-17', 'i3d-18', 'i3d-19', 'i3d-20', 'i3d-21', 'i3d-22', 'i3d-23', 'i3d-24', 'i3d-25']:
    model = get_model(name)
    val_acc, val_loss = evaluate_model(model, name)
    del model
#    models.pop(name, None)
    clear_session()
    #tf.keras.backend.clear_session()
    gc.collect()
    box_acc.append(val_acc)
    box_loss.append(val_loss)
    names.append(name)
    with open('./logs/kfold_plotdata/plotdata_box_acc_' + name + '.csv', "wb") as fp:
        pickle.dump(box_acc, fp)
    with open('./logs/kfold_plotdata/plotdata_box_loss_' + name + '.csv', "wb") as fp:
        pickle.dump(box_loss, fp)
    with open('./logs/kfold_plotdata/plotdata_names_' + name + '.csv', "wb") as fp:
        pickle.dump(names, fp)

Training fold 1 of i3d-17 with augmentation...

Epoch 00001: val_accuracy improved from -inf to 0.69408, saving model to ./logs/kfold\best_weights_fold1_i3d-17.hdf5

Epoch 00002: val_accuracy did not improve from 0.69408

Epoch 00003: val_accuracy improved from 0.69408 to 0.76042, saving model to ./logs/kfold\best_weights_fold1_i3d-17.hdf5

Epoch 00004: val_accuracy improved from 0.76042 to 0.76151, saving model to ./logs/kfold\best_weights_fold1_i3d-17.hdf5

Epoch 00005: val_accuracy did not improve from 0.76151

Epoch 00006: val_accuracy did not improve from 0.76151

Epoch 00007: val_accuracy improved from 0.76151 to 0.77632, saving model to ./logs/kfold\best_weights_fold1_i3d-17.hdf5

Epoch 00008: val_accuracy improved from 0.77632 to 0.80482, saving model to ./logs/kfold\best_weights_fold1_i3d-17.hdf5

Epoch 00009: val_accuracy did not improve from 0.80482

Epoch 00010: val_accuracy did not improve from 0.80482

Epoch 00011: val_accuracy improved from 0.80482 to 0.82072, saving mod

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1)

ax1.boxplot(box_acc, labels = names, showmeans = True)
ax2.set_title('val accuracy')
ax2.set_ylabel('accuracy')
ax2.set_xlabel('model')

ax2.boxplot(box_loss, labels = names, showmeans = True)
ax2.set_title('val loss')
ax2.set_ylabel('loss')
ax2.set_xlabel('model')

fig.savefig('./logs/kfold' + current_time + '/plot_val_acc_cv' + current_time + '.svg', format = 'svg')